In [29]:
# Install pandas
!pip3 install pandas


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [49]:
# Importing Pandas and CSV to read in the data
import pandas as pd
import csv



In [50]:
# Get 538 Ratings
ratings_538_df = pd.read_csv("MM Data/538 Ratings.csv")

In [51]:
print(len(ratings_538_df))

ratings_538_df.head(5)

476


,YEAR,TEAM NO,TEAM,SEED,ROUND,POWER RATING,POWER RATING RANK
0,2023,993,Houston,1,16,93.2,13
1,2023,1011,Alabama,1,16,92.6,16
2,2023,986,Kansas,1,32,89.6,49
3,2023,963,Purdue,1,64,89.5,50
4,2023,957,Texas,2,8,90.1,43


In [52]:
# Get some summary statistics

years_538 = set(ratings_538_df["YEAR"])
teams_538 = set(ratings_538_df["TEAM"])

print("538 Years:", years_538)
print("Number of Teams:", len(teams_538))


538 Years: {2016, 2017, 2018, 2019, 2021, 2022, 2023}
Number of Teams: 188


In [53]:
# Group by team name and look at power rating statistics ()
team_power_ratings = ratings_538_df.groupby("TEAM")["POWER RATING"].agg(
    size="size",
    mean="mean",
    median="median",
    range=lambda x: x.max() - x.min()
)

#Sort by mean
team_power_ratings = team_power_ratings.sort_values(by="mean", ascending=False)

In [54]:
team_power_ratings.head(10), team_power_ratings.tail(10)

(                size       mean  median  range
 TEAM                                          
 Gonzaga            7  92.442857   93.70   10.5
 Duke               6  90.816667   90.80    9.2
 Villanova          6  90.450000   90.75   10.4
 Virginia           6  90.450000   91.25   11.4
 Kentucky           6  89.950000   90.95    6.3
 North Carolina     6  89.916667   91.15    9.9
 Kansas             7  89.814286   90.90   11.0
 Houston            5  89.420000   88.70    7.0
 Arizona            5  89.160000   89.00    3.0
 Purdue             7  88.928571   89.00    4.6,
                         size       mean  median  range
 TEAM                                                  
 Texas Southern             5  68.940000    68.8    4.4
 Austin Peay                1  68.800000    68.8    0.0
 Texas A&M Corpus Chris     2  68.600000    68.6    2.6
 Hampton                    1  68.600000    68.6    0.0
 Southern                   1  68.000000    68.0    0.0
 Southeast Missouri St.     1  

In [55]:
# Also look at seed power ratings (Most likely seed will follow ratings (higher rating lower seed))

# Group by team name to see 
seed_power_ratings = ratings_538_df.groupby("SEED")["POWER RATING"].agg(
    size="size",
    mean="mean",
    median="median",
    range=lambda x: x.max() - x.min()
)

#Sort by mean
seed_power_ratings = seed_power_ratings.sort_values(by="mean", ascending=False)

In [56]:
seed_power_ratings.head(16)

,size,mean,median,range
SEED,,,,
1,29,92.475862,93.10,8.8
2,28,90.400000,90.70,5.0
3,27,87.414815,87.20,8.2
4,29,87.244828,86.90,8.1
5,27,86.414815,86.40,8.8
6,28,85.542857,85.65,7.3
7,28,83.678571,83.55,9.8
8,28,83.292857,83.40,9.3
9,28,82.928571,82.70,7.5


In [57]:
sum(seed_power_ratings["size"])

476

In [58]:
import pickle

# Convert aggregated statistics to dictionary

team_dict = team_power_ratings.to_dict("index")
seed_dict = seed_power_ratings.to_dict("index")

# Dump data into pickle file
with open('pickle_files/538-historic-team-data.pkl', 'wb') as team_file:
    pickle.dump(team_dict, team_file, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('pickle_files/538-historic-seed-data.pkl', 'wb') as seed_file:
    pickle.dump(seed_dict, seed_file, protocol=pickle.HIGHEST_PROTOCOL)

print(list(team_dict.items())[:5])
print(list(seed_dict.items())[:5])
print(len(team_dict))



[('Gonzaga', {'size': 7, 'mean': 92.44285714285715, 'median': 93.7, 'range': 10.5}), ('Duke', {'size': 6, 'mean': 90.81666666666666, 'median': 90.8, 'range': 9.200000000000003}), ('Villanova', {'size': 6, 'mean': 90.45, 'median': 90.75, 'range': 10.400000000000006}), ('Virginia', {'size': 6, 'mean': 90.45, 'median': 91.25, 'range': 11.400000000000006}), ('Kentucky', {'size': 6, 'mean': 89.95, 'median': 90.95, 'range': 6.299999999999997})]
[(1, {'size': 29, 'mean': 92.47586206896553, 'median': 93.1, 'range': 8.799999999999997}), (2, {'size': 28, 'mean': 90.39999999999999, 'median': 90.69999999999999, 'range': 5.0}), (3, {'size': 27, 'mean': 87.4148148148148, 'median': 87.2, 'range': 8.200000000000003}), (4, {'size': 29, 'mean': 87.2448275862069, 'median': 86.9, 'range': 8.099999999999994}), (5, {'size': 27, 'mean': 86.4148148148148, 'median': 86.4, 'range': 8.799999999999997})]
188


In [59]:
# create overall dictionary to hold 538 ratings data
overall_538_ratings_dict = dict()



# Store 538 Analysis csv in dictionary and save as pkl file
for _, row in ratings_538_df.iterrows():
    team_row = row.to_dict()
    
    overall_538_ratings_dict[(team_row["TEAM"], team_row["YEAR"])] = {
        "538 Power Rating": team_row["POWER RATING"],
        "538 Power Rank": team_row["POWER RATING RANK"]
    }
    

# Dump data into pickle file
with open('pickle_files/538-season-team-data.pkl', 'wb') as single_season_team_file:
    pickle.dump(overall_538_ratings_dict, single_season_team_file, protocol=pickle.HIGHEST_PROTOCOL)
    


